In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## HYPERPARAMETERS
block_size = 64
batch_size = 128
max_iters = 1000 # max iterations of training loop
learning_rate = 3e-4
eval_iters = 100
dropout = 0.2 # rate at which random nodes in the network are dropped so we don't overfit
n_embd = 384 # embedding_vector = [0.1,0.2,0.8,1.1,...(384 values)]
n_layer = 8  # how many decoder blocks/layer we have running sequentially
n_head = 8

In [12]:
# open text
chars = ""
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [13]:
# initialize encoder and decoder
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encode the text
data = torch.tensor(encode(text), dtype=torch.long)

In [14]:
# training and validation data split
# an analogy is imagine you're in a math course - you learn 80% of the material and the last 20% are the tests/quizzes
# it is related material you've never seen before, but you can use what you've learned (patterns) to generate something similar/alike
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# creating a stack of input and target stacks
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device) # the .to(device) would push the stacks to the cuda GPU, but I don't have one
    return x,y

# x,y = get_batch('train')
# print('inputs:')
# print(x)
# print('targets:')
# print(y)

In [15]:
@torch.no_grad() # a decorator telling PyTorch not to use gradients - reduces computation, memory usage - we're just reporting a loss, no optimization
def estimate_loss():
    out = {}
    model.eval() # model is being evaluated
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # weights and bias updated during training
    return out

In [16]:
class Head(nn.Module):
    """ one head of self attention """

    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        # input of size (batch, time-step, channels)
        # output of size(batch, time_step, head size)
        B,T,C = x.shape
        k = self.key(x) # (B,T,hs)
        q = self.query(x) # (B,T,hs)

        # compute attention scores("affinities")
        wei = q@k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,hs) @ (B,hs,T) -> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B,T,T)
        wei = F.softmax(wei,dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # perform weighted aggregation of values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B,T,T) @ (B,T,hs) -> (B,T,hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd) # adds more learnable parameters
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        # concatenating the heads together
        # (B,T,F) -> (B,T,[h1,h1,h1,h1,h2,h2,h2,h2,h3,h3,h3,h3])
        out = torch.cat([h(x) for h in self.heads], dim=-1) 
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a sample linear layer followed by a non-linearity """

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout)
        )

    def forward(self,x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self,n_embd,n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head # number of features each head captures in multi-head attention
        self.sa = MultiHeadAttention(n_head,head_size) # sa = self attention
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        y = self.sa(x) # self attention
        x = self.ln1(x+y) # add & normalize (layer normalization)
        y = self.ffwd(x) # feed forward
        x = self.ln2(x+y) # add & normalize
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        # essentially a lookup table with likelihoods of seeing a certain character in the vocab given a character in the vocab
        # given a, the likelihood of seeing a
        # given a, the likelihood of seeing b
        # given a, the likelihood of seeing c
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd) # n_embd is the size of the vector representing a character

        # other models use static sinusoidal methods, but since we're making a GPT, we use embeddings to make the values learnable
        self.position_embedding_table = nn.Embedding(block_size,n_embd) # positional embeddings of each index

        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size) # transforming output of decoders into something softmax can work with

        self.apply(self._init_weights)

    # initializing weights, giving them a standard deviation of 0.02 so that the neurons don't have outlier values throwing off learning
    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    

    def forward(self,index,targets=None):
        B,T = index.shape # batch, time, channel (vocab size)

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
        else:
            B,T,C = logits.shape
            # In PyTorch, view() allows you to reshape a tensor without changing its underlying data, essentially just reorganizing the data in memory
            logits = logits.view(B*T, C) # see comment on why we need and can do reshaping
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets) # measures how well the model’s predictions (logits) match the true labels (targets)
        
        return logits,loss

    def generate(self, index, max_new_tokens): # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits,loss = self.forward(index) # get current context
            # focus on last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits,dim=-1) # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1) # (B,1)
            # append sampled index to the running sequence
            index = torch.cat((index,index_next), dim=1) # (B,T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

# context = torch.zeros((1,1), dtype=torch.long,device=device)
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)

In [17]:
# PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate) # AdamW is an optimizer function featuring weight decay
# AdamW has momentum and adaptive learning, adjusting step size and learning rate
# AdamW also has weight decay, helping avoid overfitting the data by shrinking weights a little each time

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    # sample of batch data
    xb,yb = get_batch('train')

    # evaluate loss
    logits,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True) # only optimizing on current gradient of current data (not keeping track of old gradients)
    loss.backward()
    optimizer.step()

print(loss.item())

KeyboardInterrupt: 